In [1]:
from dataset_generation import *

In [2]:
import os
import numpy as np
import seaborn as sns
import tensorflow as tf, tf_keras
import tensorflow_hub as hub
from tensorflow.keras import layers
from tf_keras.optimizers import Adam
from tf_keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model
from official.projects.movinet.tools import export_saved_model

/Users/rebeccakrall/miniconda3/envs/tf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = 'a0'
use_positional_encoding = model_id in {'a3', 'a4', 'a5'}
resolution = 172

backbone = movinet.Movinet(
    model_id=model_id,
    causal=True,
    conv_type='2plus1d',
    se_type='2plus3d',
    activation='hard_swish',
    gating_activation='hard_sigmoid',
    use_positional_encoding=use_positional_encoding,
    use_external_states=False,
)

model = movinet_model.MovinetClassifier(
    backbone,
    num_classes=1,
    output_states=True)

# Create your example input here.
# Refer to the paper for recommended input shapes.
inputs = tf.ones([8, 30, 172, 172, 3]) #(8, 8, 224, 224, 3)

# [Optional] Build the model and load a pretrained checkpoint.
model.build(inputs.shape)





# load previous checkpoing
# checkpoint_dir = 'binary_model'
# checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
# checkpoint = tf.train.Checkpoint(model=model)
# status = checkpoint.restore(checkpoint_path)
# status.assert_existing_objects_matched()

# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  distribution_strategy = tf.distribute.experimental.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  distribution_strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  distribution_strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  distribution_strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')

print("Number of accelerators: ", distribution_strategy.num_replicas_in_sync)

2024-12-18 14:58:49.789182: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-12-18 14:58:49.789211: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-18 14:58:49.789219: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-18 14:58:49.789236: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-18 14:58:49.789249: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [4]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model


batch_size = 8
num_frames = 30
# Construct loss, optimizer and compile the model
with distribution_strategy.scope():
  model = build_classifier(batch_size, num_frames, resolution, backbone, 1)
  loss_obj =  BinaryCrossentropy(from_logits=True)
  optimizer = tf_keras.optimizers.legacy.Adam(learning_rate = 0.001)
  model.compile(loss=loss_obj, optimizer=optimizer, metrics=['accuracy'])

In [5]:
# checkpoint_dir = "/Users/rebeccakrall/Code/MoviNet Models and Training Sets/headshake_a0_172_1_60_20_072424"
checkpoint_dir = "/Users/rebeccakrall/Code/Multiclass_Behavioral_Classification/Models/scratch_120624"
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [6]:
from dataset_generation import format_frames
from video_preprocessing import get_params_from_vid, padcrop_image, frame_crop
class VidGenerator:
    def __init__(self, video_path, n_frames = 1, frame_step = 1, buffer = 300, output_size = (172,172), 
                 animal_choice = ['m1'], dp = 0.5, minDist = 400, start = 0, stop = None):
        """ Returns a set of frames with their associated label. 

        Args:
            df_path: path to dataframe
            indices: indices of df to use for 
            n_frames: Number of frames. 
            training: Boolean to determine if training dataset is being created.
        """
        self.video_path = video_path
        self.batch_size = n_frames * frame_step
        self.buffer = buffer
        self.output_size = output_size
        circles, self.vid_frames, self.fps, _ = get_params_from_vid(self.video_path, dp = dp, minDist = minDist)
        self.circles = circles[animal_choice]
        self.step = frame_step
        self.start = start
        if stop is None:
            self.stop = self.vid_frames
        else:
            self.stop = stop

    def __call__(self):

        cap = cv2.VideoCapture(str(self.video_path))
        cap.set(cv2.CAP_PROP_POS_FRAMES, self.start)
        print('opened cap')
        
        for _ in range(int((self.stop-self.start)/self.batch_size)):
            result = []
            for i in range(self.batch_size):
                ret, frame = cap.read()
                if i % self.step == 0:

                    center = self.circles

                    cc, center = frame_crop(center[0], center[1], buffer = self.buffer, shape = frame.shape[:2])
                    pc = padcrop_image(frame, self.buffer, cc, center)
                    result.append(format_frames(pc, self.output_size))
                    
                    # center = self.circles # test the output of get_params_from_vide
                    # frame = padcrop_image(frame, self.buffer, center[:2])
                    # result.append(format_frames(frame, self.output_size))
            
            result = np.array(result)
            yield result[tf.newaxis,...]
            
        
        cap.release()

In [7]:
folder = "/Users/rebeccakrall/Desktop/Behavior Clips"
classes = {'Scratch':['Right Foot Scratch', 'Left Foot Scratch'],'Not Scratch': ['Face Grooming', 'Body Groom Left', 'Body Groom Right', 'Walking', 
                                                                                 'Turn Left', 'Turn Right', 'Rearing', 'Rear On Wall', 'Dog Shake', 
                                                                                 'Sleep', 'Not Scratch', 'not_scratch']}


In [8]:
scratch_files = len([name for x in classes['Scratch'] for name in os.listdir(os.path.join(folder, x)) ])
nonscratch_files = len([name for x in classes['Not Scratch'] for name in os.listdir(os.path.join(folder, x)) ])


In [9]:
scratch_files == 807 + 44 + 45


False

In [15]:
splits = {'test': 45, 'val': 44, 'train': 807}
training_splits = choose_behavior_subset(folder, classes, splits)

In [16]:
num_frames = 20
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds2 = tf.data.Dataset.from_generator(FrameGenerator(training_splits['train'],  num_frames, frame_step = 1, training = True),
                                          output_signature = output_signature)

batch_size = 8
train_ds2= train_ds2.batch(batch_size)

test_ds2 = tf.data.Dataset.from_generator(FrameGenerator(training_splits['test'], num_frames, frame_step = 1,training= True),
                                          output_signature = output_signature)
test_ds2 = test_ds2.batch(batch_size)

val_ds2 = tf.data.Dataset.from_generator(FrameGenerator(training_splits['val'], num_frames, frame_step = 1, training = True),
                                          output_signature = output_signature)
val_ds2 = val_ds2.batch(batch_size)

In [17]:
checkpoint_path = "Models/scratch_121824/cp_01.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf_keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_best_only = True)

In [18]:
history = model.fit(train_ds2,
                    validation_data=val_ds2,
                    epochs=5,
                    validation_freq=1,
                    verbose=1,
                    callbacks = [cp_callback])

Epoch 1/5
    202/Unknown - 2186s 11s/step - loss: 0.0278 - accuracy: 0.9913

2024-12-19 09:55:19.933019: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-12-19 09:55:19.933281: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10563838165649432782
2024-12-19 09:55:19.933288: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[movinet_classifier_1/movinet/block2_layer2/bneck/se/global_average_pool3d_6/Cumsum/_110]]
2024-12-19 09:55:19.933310: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 5723022188223947692
2024-12-19 09:55:19.933321: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14329843108485063489
2024-12-19 09:55:19.933331: I tensorflow/core/framework/local_rendezvous.cc:422] Local re


Epoch 1: saving model to Models/scratch_121824/cp_01.ckpt
202/202 [==============================] - 2278s 11s/step - loss: 0.0278 - accuracy: 0.9913 - val_loss: 0.0153 - val_accuracy: 1.0000
Epoch 2/5
202/202 [==============================] - ETA: 0s - loss: 0.0173 - accuracy: 0.9926 
Epoch 2: saving model to Models/scratch_121824/cp_01.ckpt
202/202 [==============================] - 2269s 11s/step - loss: 0.0173 - accuracy: 0.9926 - val_loss: 0.0297 - val_accuracy: 0.9886
Epoch 3/5
202/202 [==============================] - ETA: 0s - loss: 0.0206 - accuracy: 0.9919 
Epoch 3: saving model to Models/scratch_121824/cp_01.ckpt
202/202 [==============================] - 2270s 11s/step - loss: 0.0206 - accuracy: 0.9919 - val_loss: 0.0885 - val_accuracy: 0.9773
Epoch 4/5
202/202 [==============================] - ETA: 0s - loss: 0.0214 - accuracy: 0.9919 
Epoch 4: saving model to Models/scratch_121824/cp_01.ckpt
202/202 [==============================] - 2271s 11s/step - loss: 0.0214 - ac

In [19]:
model.evaluate(test_ds2)

     12/Unknown - 84s 7s/step - loss: 0.0233 - accuracy: 0.9889

2024-12-19 12:31:30.381958: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-12-19 12:31:30.381979: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[movinet_classifier_1/movinet/block2_layer2/bneck/se/global_average_pool3d_6/Cumsum/_62]]
2024-12-19 12:31:30.381988: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 18110576932241315612
2024-12-19 12:31:30.381992: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9816012139080753433
2024-12-19 12:31:30.382007: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4371672062624010977
2024-12-19 12:31:30.382011: I tensorflow/core/framework/local_rendezvous.cc:422] Local rend

12/12 [==============================] - 86s 7s/step - loss: 0.0233 - accuracy: 0.9889


[0.023275116458535194, 0.9888888597488403]

In [ ]:
{'Scratch':1}